# Ear Record Generator

In [2]:
%matplotlib inline

import sys
sys.path.append('/homes/yz4009/wd/gitdev/TFNet/')

import numpy as np
import menpo.io as mio
import scipy.io as sio
from io import BytesIO
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import xml.etree.ElementTree as ET
import json
import glob
import cv2
import scipy
import utils
import os
from menpo.image import Image
from menpo.visualize import print_dynamic, print_progress

from dAAMs.lineerror import interpolate
from dAAMs.tools import loadmatToDict, multi_channel_svs
from scipy.spatial.distance import pdist
from pathlib import Path

from menpo.shape import PointCloud, PointUndirectedGraph
from menpo.transform import Translation
from menpofit.transform import DifferentiableAlignmentSimilarity

from menpowidgets import visualize_images, visualize_pointclouds
from IPython.html.widgets import interact
from IPython.html.widgets import Button
from IPython.display import display, clear_output
from dAAMs.svs import SVS, MultiSVS
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import data_provider
import tensorflow as tf
slim = tf.contrib.slim


def get_jpg_string(im):
    # Gets the serialized jpg from a menpo `Image`.
    fp = BytesIO()
    mio.export_image(im, fp, extension='jpg')
    fp.seek(0)
    return fp.read()

def _int_feauture(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feauture(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feauture(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))



/vol/atlas/homes/yz4009/miniconda/envs/gitdev/lib/python3.5/site-packages/scipy/sparse/compressed.py:730: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)
/vol/atlas/homes/yz4009/gitdev/menpofit/menpofit/math/fft_utils.py:20: RuntimeWarning: pyfftw is known to be buggy on your system, numpy.fft will be used instead. Consequently, all algorithms using ffts will be running at a slower speed.
  RuntimeWarning)
/vol/atlas/homes/yz4009/miniconda/envs/gitdev/lib/python3.5/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


### Ear Record

In [4]:
store_path = Path('/homes/yz4009/wd/databases/tfrecords')
load_path = Path('/vol/atlas/databases/ear/UERC/UERC 2017 Dataset/Train Dataset')
record_name = '%s.tfrecords'%'UERC_train'

print(record_name)

def data_iterator():
    database_path = load_path

    id_no = 1
    for identity_path in print_progress(list(load_path.glob('*'))):
        if identity_path.is_dir():
            images = mio.import_images(identity_path)
            for img in images:
                cimgs = utils.crop_image(img, img.centre(), img.diagonal()/350, [256,256], base=384)[0]
                img_height = 256
                img_width = 256
                id_no = int(identity_path.stem)

                yield cimgs, img_height, img_width, id_no
                
            id_no += 1
    

def generate(iterator,
             store_path=store_path,
             record_name=record_name,
             base=384):

    store_path = Path(store_path)
    writer = tf.python_io.TFRecordWriter(str(store_path/record_name))
    
    for img_all, img_height, img_width,  id_no in iterator:

        example = tf.train.Example(
            features=tf.train.Features(
              # Features contains a map of string to Feature proto objects
                feature={
                    # images
                    'image': _bytes_feauture(get_jpg_string(img_all)),
                    'height': _int_feauture(img_height),
                    'width': _int_feauture(img_width),
                    'id_no': _int_feauture(id_no)
                }))

        # use the proto object to serialize the example to a string
        serialized = example.SerializeToString()
        # write the serialized object to disk
        writer.write(serialized)

    writer.close()

generate(data_iterator())

UERC_train.tfrecords


/vol/atlas/homes/yz4009/gitdev/menpo/menpo/image/boolean.py:553: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  boolean_image = BooleanImage(warped.pixels.reshape(template_shape))


[====================] 100% (166/166) - done.                                   


#### save to file

In [22]:
store_path = Path('/homes/yz4009/wd/databases/UERC_160')
load_path = Path('/vol/atlas/databases/ear/UERC/UERC 2017 Dataset/Train Dataset')
record_name = '%s.tfrecords'%'UERC_train'

print(record_name)

def data_iterator():
    database_path = load_path

    id_no = 1
    for identity_path in print_progress(list(load_path.glob('*'))):
        if identity_path.is_dir():
            images = mio.import_images(identity_path)
            for img_id,img in enumerate(images):
                cimgs = utils.crop_image(img, img.centre(), img.diagonal()/350, [160,160], base=384)[0]
                img_height = 160
                img_width = 160
                id_no = int(identity_path.stem)

                yield cimgs, img_height, img_width, id_no, img_id
                
            id_no += 1
    

def generate(iterator,
             store_path=store_path,
             record_name=record_name,
             base=384):

    store_path = Path(store_path)
    
    for img_all, img_height, img_width,  id_no, img_id in iterator:
        d_path = str(store_path/str(id_no))
        if not os.path.exists(d_path):
            os.mkdir(d_path)
        mio.export_image(img_all, store_path/str(id_no)/('%04d.png'%img_id))
        

generate(data_iterator())

UERC_train.tfrecords


/vol/atlas/homes/yz4009/gitdev/menpo/menpo/image/boolean.py:553: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  boolean_image = BooleanImage(warped.pixels.reshape(template_shape))


[====================] 100% (166/166) - done.                                   


In [63]:
store_path = Path('/homes/yz4009/wd/databases/UERC_160_generate')
load_path = Path('/vol/atlas/databases/ear/UERC/UERC 2017 Dataset/Test Dataset')
record_name = '%s.tfrecords'%'UERC_test'

print(record_name)

def data_iterator():
    database_path = load_path

    for identity_path in print_progress(list(load_path.glob('*'))):
        if identity_path.is_dir():
            images = mio.import_images(identity_path)
            for img_id,img in enumerate(images):
                cimgs = utils.crop_image(img, img.centre(), img.diagonal()/350, [160,160], base=384)[0]
                img_height = 160
                img_width = 160
                id_no = identity_path.stem
                image_name = img.path.name

                yield cimgs, img_height, img_width, id_no, image_name
            
    

def generate(iterator,
             store_path=store_path,
             record_name=record_name,
             base=384):

    store_path = Path(store_path)
    
    for img_all, img_height, img_width,  id_no, image_name in iterator:
        d_path = str(store_path/str(id_no))
        if not os.path.exists(d_path):
            os.mkdir(d_path)
        mio.export_image(img_all, store_path/str(id_no)/image_name)
        

generate(data_iterator())

UERC_test.tfrecords
[                    ] 1% (58/3542) - 00:03:19 remaining                        

/vol/atlas/homes/yz4009/gitdev/menpo/menpo/image/boolean.py:553: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  boolean_image = BooleanImage(warped.pixels.reshape(template_shape))


[====================] 100% (3542/3542) - done.                                 


#### vgg ear

In [61]:
store_path = Path('/homes/yz4009/wd/databases/VGGEAR_160')
load_path = Path('/homes/yz4009/wd/databases/ear/VGGEers-Recognition')

def data_iterator():
    database_path = load_path

    id_no = 167
    for identity_path in print_progress(list(load_path.glob('*'))):
        if identity_path.is_dir():
            images = mio.import_images(identity_path)
            for img_id,img in enumerate(images):
                img = img.crop_to_landmarks_proportion(0.1)
                
                if img.n_channels == 1:
                    img = Image(np.stack([img.pixels.squeeze() for _ in range(3)]))
                    
                cimgs = utils.crop_image(img, img.centre(), img.diagonal()/350, [160,160], base=384)[0]
                img_height = 160
                img_width = 160

                yield cimgs, img_height, img_width, id_no, img_id
                
            id_no += 1
    

def generate(iterator,
             store_path=store_path,
             record_name=record_name,
             base=384):

    store_path = Path(store_path)
    
    for img_all, img_height, img_width,  id_no, img_id in iterator:
        d_path = str(store_path/str(id_no))
        if not os.path.exists(d_path):
            os.mkdir(d_path)
        mio.export_image(img_all, store_path/str(id_no)/('%d_%04d.png'%(id_no,img_id)))
        

generate(data_iterator())

[                    ] 4% (10/232) - 00:00:55 remaining                         

/vol/atlas/homes/yz4009/gitdev/menpo/menpo/image/boolean.py:553: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  boolean_image = BooleanImage(warped.pixels.reshape(template_shape))


[====================] 100% (232/232) - done.                                   


### Face Record

#### train

In [18]:
np.random.choice(3,5)

array([1, 1, 2, 0, 0])

In [19]:
store_path = Path('/homes/yz4009/wd/databases/tfrecords')
load_path = Path('/vol/atlas/homes/jiankang/code/facenet/data/CASIA_182_multi/')
record_name = '%s.tfrecords'%'CASIA_182'

print(record_name)

def data_iterator():
    database_path = load_path

    image_id = 1
    for tpath in print_progress(list(load_path.glob('*'))):
        
        if tpath.is_dir():
        
            img_height = 182
            img_width = 182

            img_all = np.stack([img.pixels_with_channels_at_back() for img in mio.import_images(tpath)])
            
            if len(img_all) < 16:
                img_all = img_all[np.random.choice(len(img_all),16)]

            n_img = np.min([len(img_all), 354])

            img_all = img_all.reshape(-1,img_height,img_width,3)
            img_all = img_all[:n_img].reshape(-1,img_width,3)

            yield Image.init_from_channels_at_back(img_all), img_height, img_width, n_img, image_id

            image_id += 1
    

def generate(iterator,
             store_path=store_path,
             record_name=record_name,
             base=384):

    store_path = Path(store_path)
    writer = tf.python_io.TFRecordWriter(str(store_path/record_name))
    
    for img_all, img_height, img_width, n_img, id_no in iterator:
        try:
            example = tf.train.Example(
                features=tf.train.Features(
                  # Features contains a map of string to Feature proto objects
                    feature={
                        # images
                        'image': _bytes_feauture(get_jpg_string(img_all)),
                        'height': _int_feauture(img_height),
                        'width': _int_feauture(img_width),
                        'n_image': _int_feauture(n_img),
                        'id_no': _int_feauture(id_no)
                    }))

            # use the proto object to serialize the example to a string
            serialized = example.SerializeToString()
            # write the serialized object to disk
            writer.write(serialized)
        except Exception as e:
            print(e)


    writer.close()

generate(data_iterator())

CASIA_182.tfrecords
[====================] 100% (10580/10580) - done.                               


In [49]:
store_path = Path('/homes/yz4009/wd/databases/tfrecords')
load_path = Path('/vol/atlas/homes/jiankang/data/recognition/data/CASIA_112/')
record_name = '%s.tfrecords'%'CASIA'

print(record_name)

def data_iterator():
    database_path = load_path

    for timg in print_progress(mio.import_images(load_path)):
        img_height = 112
        img_width = 112
        id_no = int(timg.path.stem)
        
        nh,nw = np.array(timg.shape) // 112
        
        n_img = np.min([nh*nw, 36*16])
        img_all = timg.pixels.reshape(3,nh,112,nw,112).transpose(1,3,2,4,0).reshape(-1,112,112,3)
        img_all = img_all[:n_img].reshape(-1,112,3)
        
        yield Image.init_from_channels_at_back(img_all), img_height, img_width, n_img, id_no
    

def generate(iterator,
             store_path=store_path,
             record_name=record_name,
             base=384):

    store_path = Path(store_path)
    writer = tf.python_io.TFRecordWriter(str(store_path/record_name))
    
    for img_all, img_height, img_width, n_img, id_no in iterator:
        try:
            example = tf.train.Example(
                features=tf.train.Features(
                  # Features contains a map of string to Feature proto objects
                    feature={
                        # images
                        'image': _bytes_feauture(get_jpg_string(img_all)),
                        'height': _int_feauture(img_height),
                        'width': _int_feauture(img_width),
                        'n_image': _int_feauture(n_img),
                        'id_no': _int_feauture(id_no)
                    }))

            # use the proto object to serialize the example to a string
            serialized = example.SerializeToString()
            # write the serialized object to disk
            writer.write(serialized)
        except Exception as e:
            print(e)


    writer.close()

generate(data_iterator())

CASIA.tfrecords
[====================] 100% (10575/10575) - done.                               


#### evaluate

In [ ]:
store_path = Path('/homes/yz4009/wd/databases/tfrecords')
load_path = Path('/vol/atlas/homes/jiankang/code/facenet/data/lfw_160/')
record_name = '%s.tfrecords'%'LFW_160'

print(record_name)

def data_iterator():
    database_path = load_path

    image_id = 1
    for tpath in print_progress(list(load_path.glob('*'))):
        
        if tpath.is_dir():
        
            img_height = 182
            img_width = 182

            img_all = np.stack([img.pixels_with_channels_at_back() for img in mio.import_images(tpath)])

            n_img = np.min([len(img_all), 354])

            img_all = img_all.reshape(-1,img_height,img_width,3)
            img_all = img_all[:n_img].reshape(-1,img_width,3)

            yield Image.init_from_channels_at_back(img_all), img_height, img_width, n_img, image_id

            image_id += 1
    

def generate(iterator,
             store_path=store_path,
             record_name=record_name,
             base=384):

    store_path = Path(store_path)
    writer = tf.python_io.TFRecordWriter(str(store_path/record_name))
    
    for img_all, img_height, img_width, n_img, id_no in iterator:
        try:
            example = tf.train.Example(
                features=tf.train.Features(
                  # Features contains a map of string to Feature proto objects
                    feature={
                        # images
                        'image': _bytes_feauture(get_jpg_string(img_all)),
                        'height': _int_feauture(img_height),
                        'width': _int_feauture(img_width),
                        'n_image': _int_feauture(n_img),
                        'id_no': _int_feauture(id_no)
                    }))

            # use the proto object to serialize the example to a string
            serialized = example.SerializeToString()
            # write the serialized object to disk
            writer.write(serialized)
        except Exception as e:
            print(e)


    writer.close()

generate(data_iterator())

In [77]:
store_path = Path('/homes/yz4009/wd/databases/tfrecords')
load_path = Path('/vol/atlas/homes/jiankang/data/recognition/data/lfw_112/')
record_name = '%s.tfrecords'%'LFW'

print(record_name)

def data_iterator():
    database_path = load_path
    
    img_height = 112
    img_width = 112
    n_img=2
    
    with open('/homes/yz4009/Desktop/lfw_pairs.txt') as f:
        pairs = f.readlines()
        n_fold, n_pairs = map(int, pairs[0].strip().split('\t'))
        pairs = pairs[1:]
        for fold in print_progress(range(n_fold)):
            for p in range(n_pairs):
                name,id1,id2=pairs[fold*n_pairs*2+p].strip().split('\t')
                img1 = mio.import_image(database_path/name/('%s_%04d.jpg'%(name, int(id1))))
                img2 = mio.import_image(database_path/name/('%s_%04d.jpg'%(name, int(id2))))
                img_all = Image(np.concatenate([img1.pixels, img2.pixels], axis=1))
                
                yield img_all, img_height, img_width, n_img, 1

            for p in range(n_pairs, n_pairs*2):
                name1,id1, name2,id2=pairs[fold*n_pairs*2+p].strip().split('\t')
                img1 = mio.import_image(database_path/name1/('%s_%04d.jpg'%(name1, int(id1))))
                img2 = mio.import_image(database_path/name2/('%s_%04d.jpg'%(name2, int(id2))))
                img_all = Image(np.concatenate([img1.pixels, img2.pixels], axis=1))
                
                yield img_all, img_height, img_width, n_img, 0
    

def generate(iterator,
             store_path=store_path,
             record_name=record_name,
             base=384):

    store_path = Path(store_path)
    writer = tf.python_io.TFRecordWriter(str(store_path/record_name))
    
    for img_all, img_height, img_width, n_img, id_no in iterator:
        try:
            example = tf.train.Example(
                features=tf.train.Features(
                  # Features contains a map of string to Feature proto objects
                    feature={
                        # images
                        'image': _bytes_feauture(get_jpg_string(img_all)),
                        'height': _int_feauture(img_height),
                        'width': _int_feauture(img_width),
                        'n_image': _int_feauture(n_img),
                        'id_no': _int_feauture(id_no)
                    }))

            # use the proto object to serialize the example to a string
            serialized = example.SerializeToString()
            # write the serialized object to disk
            writer.write(serialized)
        except Exception as e:
            print(e)


    writer.close()

generate(data_iterator())

LFW.tfrecords
[====================] 100% (10/10) - done.                                     


In [1]:
import struct